In [ ]:
import numpy as np
import pywt
import pickle
from PIL import Image
import os
import heapq
import time
import csv


Huffman

In [ ]:

class node:
    def __init__(self, frequency, symbol, left=None, right=None):
        self.frequency = frequency
        self.symbol = symbol
        self.left = left
        self.right = right
        self.huffman_direction = ''

    def __lt__(self, nxt):
        return self.frequency < nxt.frequency

def get_frequency(bit_string):
    byte_to_frequency = {}
    for i in range(0, len(bit_string), 8):
        byte = bit_string[i:i + 8]
        if byte not in byte_to_frequency:
            byte_to_frequency[byte] = 0
        byte_to_frequency[byte] += 1
    return byte_to_frequency

def get_merged_huffman_tree(byte_to_frequency):
    huffman_tree = []
    for byte, frequency in byte_to_frequency.items():
        heapq.heappush(huffman_tree, node(frequency, byte))
    while len(huffman_tree) > 1:
        left = heapq.heappop(huffman_tree)
        right = heapq.heappop(huffman_tree)
        left.huffman_direction = "0"
        right.huffman_direction = "1"
        merged_node = node(left.frequency + right.frequency, left.symbol + right.symbol, left, right)
        heapq.heappush(huffman_tree, merged_node)
    return huffman_tree[0]

def calculate_huffman_codes(node, code='', huffman_codes={}):
    code += node.huffman_direction
    if node.left:
        calculate_huffman_codes(node.left, code, huffman_codes)
    if node.right:
        calculate_huffman_codes(node.right, code, huffman_codes)
    if not node.left and not node.right:
        huffman_codes[node.symbol] = code
    return huffman_codes

def compress(bit_string):
    byte_to_frequency = get_frequency(bit_string)
    merged_tree = get_merged_huffman_tree(byte_to_frequency)
    huffman_codes = calculate_huffman_codes(merged_tree)
    compressed = ''.join(huffman_codes[bit_string[i:i+8]] for i in range(0, len(bit_string), 8))
    return compressed, huffman_codes

def decompress(compressed_string, huffman_codes):
    reverse_codes = {v: k for k, v in huffman_codes.items()}
    current_code = ""
    decompressed = ""
    for bit in compressed_string:
        current_code += bit
        if current_code in reverse_codes:
            decompressed += reverse_codes[current_code]
            current_code = ""
    return decompressed

def write_binary(bit_string, path):
    with open(path, 'wb') as f:
        for i in range(0, len(bit_string), 8):
            byte = bit_string[i:i+8]
            f.write(bytes([int(byte, 2)]))

def read_binary(path):
    with open(path, 'rb') as f:
        return ''.join(format(byte, '08b') for byte in f.read())



DWT

In [ ]:

def compress_ecg_image(img_array, wavelet='haar', level=2, threshold=10):
    coeffs = pywt.wavedec2(img_array, wavelet=wavelet, level=level)
    coeff_arr, coeff_slices = pywt.coeffs_to_array(coeffs)
    coeff_arr[np.abs(coeff_arr) < threshold] = 0
    return coeff_arr, coeff_slices

def decompress_ecg_image(coeff_arr, coeff_slices, wavelet='haar'):
    coeffs_from_arr = pywt.array_to_coeffs(coeff_arr, coeff_slices, output_format='wavedec2')
    reconstructed = pywt.waverec2(coeffs_from_arr, wavelet=wavelet)
    return np.clip(reconstructed, 0, 255).astype(np.uint8)



Full pipeline

In [ ]:
durations = []
def compress_and_reconstruct_folder(input_folder, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    def bin16_to_signed_int(b):
        val = int(b, 2)
        if val >= 2**15:
            val -= 2**16
        return val

    for filename in os.listdir(input_folder):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            continue
        start_time = time.time()
        input_path = os.path.join(input_folder, filename)
        image = Image.open(input_path).convert('L')
        arr = np.array(image)

        # Compression
        coeff_arr, coeff_slices = compress_ecg_image(arr)
        quantized = np.round(coeff_arr).astype(np.int16)
        flat = quantized.flatten()
        bit_string = ''.join(format(val & 0xFFFF, '016b') for val in flat)
        compressed_bits, codes = compress(bit_string)

        # Decompression
        decompressed_bits = decompress(compressed_bits, codes)
        flat_vals = [bin16_to_signed_int(decompressed_bits[i:i+16]) for i in range(0, len(decompressed_bits), 16)]
        reshaped = np.array(flat_vals, dtype=np.int16).reshape(quantized.shape)
        reconstructed = decompress_ecg_image(reshaped, coeff_slices)

        # Save reconstructed image
        save_path = os.path.join(output_folder, filename)
        Image.fromarray(reconstructed).save(save_path)
        end_time = time.time()
        duration = end_time - start_time
        durations.append((filename, duration))
    csv_path = 'dwt_huffman.csv'
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Image Name', 'Duration (seconds)'])
        writer.writerows(durations)
    
    print(f"✅ Duration tracking complete. Saved to: {csv_path}")

In [11]:
compress_and_reconstruct_folder("org_ecg_10sec_resize", "dwt_huffman")

✅ Duration tracking complete. Saved to: dwt_huffman.csv
